In [2]:
pip install opencv-python pyzbar pillow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import numpy as np
from pyzbar.pyzbar import decode
import sqlite3
from tkinter import *
from tkinter import messagebox
from PIL import Image, ImageTk
import threading
import webbrowser
import base64
from io import BytesIO
import time
import os

# Initialize database
conn_lock = threading.Lock()

def get_connection():
    try:
        return sqlite3.connect('qr_history.db')
    except sqlite3.Error as e:
        print(f"Database connection error: {e}")
        return None

def initialize_db():
    try:
        conn = get_connection()
        if conn is not None:
            with conn_lock:
                c = conn.cursor()
                c.execute('''CREATE TABLE IF NOT EXISTS history (
                                id INTEGER PRIMARY KEY AUTOINCREMENT,
                                data TEXT NOT NULL)''')
                conn.commit()
                conn.close()
    except sqlite3.Error as e:
        print(f"Database initialization error: {e}")

# Attempt to initialize the database
initialize_db()

# Function to save history
def save_history(data):
    try:
        with conn_lock:
            conn = get_connection()
            if conn is not None:
                c = conn.cursor()
                c.execute("INSERT INTO history (data) VALUES (?)", (data,))
                conn.commit()
                conn.close()
            else:
                print("Failed to connect to the database.")
    except sqlite3.Error as e:
        print(f"Error saving to database: {e}")

# Function to get history
def get_history():
    try:
        with conn_lock:
            conn = get_connection()
            if conn is not None:
                c = conn.cursor()
                c.execute("SELECT * FROM history")
                result = c.fetchall()
                conn.close()
                return result
            else:
                print("Failed to connect to the database.")
                return []
    except sqlite3.Error as e:
        print(f"Error retrieving from database: {e}")
        return []

# Function to clear history
def clear_history():
    try:
        with conn_lock:
            conn = get_connection()
            if conn is not None:
                c = conn.cursor()
                c.execute("DELETE FROM history")
                conn.commit()
                conn.close()
            else:
                print("Failed to connect to the database.")
    except sqlite3.Error as e:
        print(f"Error clearing database history: {e}")

# Function to scan QR code
def scan_qr(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    decoded_objects = decode(gray)
    for obj in decoded_objects:
        points = obj.polygon
        if len(points) > 4:
            hull = cv2.convexHull(np.array([point for point in points], dtype=np.float32))
            points = hull
        n = len(points)
        for j in range(0, n):
            cv2.line(frame, tuple(points[j]), tuple(points[(j + 1) % n]), (0, 255, 0), 3)
        qr_data = obj.data.decode('utf-8')
        cv2.putText(frame, qr_data, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        return qr_data
    return None

class QRScannerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("QR Code Scanner")
        self.root.configure(bg='#282c34')  # Background color

        self.label = Label(root, bg='#282c34')
        self.label.pack()

        self.btn_frame = Frame(root, bg='#282c34')
        self.btn_frame.pack(pady=10)

        self.start_btn = Button(self.btn_frame, text="Start Scanning", command=self.start_scanning, bg='#61afef', fg='white', font=('Arial', 12, 'bold'))
        self.start_btn.pack(side=LEFT, padx=10)

        self.stop_btn = Button(self.btn_frame, text="Stop Scanning", command=self.stop_scanning, bg='#e06c75', fg='white', font=('Arial', 12, 'bold'))
        self.stop_btn.pack(side=LEFT, padx=10)

        self.history_btn = Button(self.btn_frame, text="View History", command=self.view_history, bg='#98c379', fg='white', font=('Arial', 12, 'bold'))
        self.history_btn.pack(side=LEFT, padx=10)

        self.clear_history_btn = Button(self.btn_frame, text="Clear History", command=self.clear_history, bg='#e06c75', fg='white', font=('Arial', 12, 'bold'))
        self.clear_history_btn.pack(side=LEFT, padx=10)

        self.cap = None
        self.running = False
        self.thread = None

    def start_scanning(self):
        if not self.running:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                messagebox.showerror("Error", "Unable to access the camera.")
                return
            self.running = True
            self.thread = threading.Thread(target=self.scan)
            self.thread.start()

    def scan(self):
        while self.running:
            ret, frame = self.cap.read()
            if ret:
                qr_data = scan_qr(frame)
                if qr_data:
                    save_history(qr_data)
                    self.running = False
                    self.handle_qr_data(qr_data)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(frame)
                imgtk = ImageTk.PhotoImage(image=img)
                self.label.imgtk = imgtk
                self.label.configure(image=imgtk)
            else:
                self.stop_scanning()

            # Add delay to manage frame rate and avoid flickering
            time.sleep(0.05)

    def stop_scanning(self):
        self.running = False
        if self.cap:
            self.cap.release()
            self.cap = None  # Ensure the camera is released
        # Wait for the scanning thread to finish if it's still running
        if self.thread and self.thread.is_alive():
            self.thread.join()

    def handle_qr_data(self, qr_data):
        if qr_data.startswith('http://') or qr_data.startswith('https://'):
            # If the data is a URL, open it in a web browser
            webbrowser.open(f"https://www.google.com/search?q={qr_data}")
        elif qr_data.startswith('data:image/'):
            # If the data is a base64 encoded image
            header, encoded = qr_data.split(",", 1)
            data = base64.b64decode(encoded)
            image = Image.open(BytesIO(data))
            self.display_image(image)
        else:
            messagebox.showinfo("QR Data", qr_data)

    def display_image(self, image):
        img_window = Toplevel(self.root)
        img_window.title("QR Image")
        img_window.configure(bg='#282c34')  # Background color
        img = ImageTk.PhotoImage(image)
        img_label = Label(img_window, image=img, bg='#282c34')
        img_label.image = img  # Keep a reference to avoid garbage collection
        img_label.pack()

    def view_history(self):
        self.stop_scanning()
        history = get_history()
        history_window = Toplevel(self.root)
        history_window.title("Scan History")
        history_window.configure(bg='#282c34')  # Background color
        for row in history:
            Label(history_window, text=row[1], bg='#282c34', fg='white', font=('Arial', 12)).pack(pady=2)

    def clear_history(self):
        clear_history()
        messagebox.showinfo("Info", "History Cleared")

    def on_closing(self):
        self.stop_scanning()
        self.root.destroy()

if __name__ == "__main__":
    root = Tk()
    app = QRScannerApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)
    root.mainloop()


Database initialization error: file is not a database
Error retrieving from database: file is not a database
